##  [망고플레이트 크롤링](https://www.mangoplate.com/)    
- **페이지 1개씩 크롤링**   
- 참고 사이트 : https://juwon2021.tistory.com/414

### ** 시작 전 확인 사항 **
  Window terminal에서   
- conda activate   
- conda activate ml-env   
(ml-env) 환경 확인 후   
- jupyter notebook   
- conda install -c anaconda requests   
- conda install bs4   
- conda install lxml   
- conda install -c conda-forge selenium   
- conda install tqdm   
- connda install -c conda-forge folium   
- connda install -c conda-forge googlemaps   
- pip install chromedriver-autoinstaller   

### 페이지 1개씩 크롤링 (for문 활용)

In [1]:
# 필요한 라이브러리를 임포트하기

import sys # 시스템
import os  # 시스템

# 데이터 다루기
import pandas as pd
import numpy as np

# selenium 크롤링
from selenium import webdriver  
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 크롬 드라이버
import chromedriver_autoinstaller

# beautifulsoup 크롤링
import requests
from bs4 import BeautifulSoup

# lxml 크롤링
import lxml.html

# 시간 조절
import time

# 시간 측정
from tqdm import notebook

# 정규표현식
import re

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 지역 키워드를 입력하기
keyword = '강남'

# page 정하기
page = 1

In [3]:
# 크롬창 띄우기
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)
driver.maximize_window() # 창 최대화

# 키워드가 입력된 망고플레이트 사이트에 들어가기
driver.get(" https://www.mangoplate.com/search/{0}?keyword={0}&page={1}".format(keyword, page)) 
time.sleep(2)

In [4]:
# 음식점 1개에서 정보 크롤링 함수
def FoodCrawling(row, col):
    # 전체 담을 그릇
    total_dict = {}
    k = 0
    
    for i in range(row):
        driver.execute_script("window.scrollTo(0, 200)")
        for j in range(col):
            # 동영상 1개 담을 그릇
            sub_dict = {}
            
            # 음식점 클릭
            element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > a > div > img'
            driver.find_element(By.CSS_SELECTOR, element).click()
            time.sleep(0.5)

            # 가게 이름 크롤링
            element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1'
            title_raw = driver.find_element(By.CSS_SELECTOR, element)
            title = title_raw.text

            # 가게 전체 평점 점수 크롤링
            element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span'
            total_raw = driver.find_element(By.CSS_SELECTOR, element)
            total = total_raw.text
            
            # 조회수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.hit'
                view_raw = driver.find_element(By.CSS_SELECTOR, element)
                view = view_raw.text
            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.hit'
                view_raw = driver.find_element(By.CSS_SELECTOR, element)
                view = view_raw.text

            # 리뷰 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.review'
                review_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_review = review_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.review'
                review_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_review = review_raw.text

            # 별표 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.favorite'
                star_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_star = star_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.favorite'
                star_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_star = star_raw.text

            # 음식 종류 및 메뉴 크롤링
            element1 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(3) > td > span'
            type_raw = driver.find_element(By.CSS_SELECTOR, element1)
            type_raw = type_raw.text
            food_type = type_raw
            try:
                element2 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(9) > td > ul'
                menu_raw = driver.find_element(By.CSS_SELECTOR, element2)
                food_menu = menu_raw.text
                food_type = [type_raw, food_menu]
            except:
                pass
            
            # sub_dict에 담기
            sub_dict['title'] = title
            sub_dict['Point'] = total
            sub_dict['View'] = view
            sub_dict['Review'] = num_review
            sub_dict['Star'] = num_star
            sub_dict['Type'] = food_type
            
             # total_dict에 담기
            total_dict[k] = sub_dict
            k += 1
            time.sleep(1)
            
            # 뒤로 가기
            driver.back()
            time.sleep(0.5)
            
    df = pd.DataFrame.from_dict(total_dict, orient='index')
    driver.close()
    return df

In [5]:
df = FoodCrawling(row=10, col=2)
df

,title,Point,View,Review,Star,Type
0,갓포아키,4.6,"139,661",52,"2,766",이자카야 / 오뎅 / 꼬치
1,호시쿠,4.6,"14,063",18,429,회 / 스시
2,고메램,4.6,"80,499",47,"1,381","[스테이크 / 바베큐, 모둠 양갈비\n33,000원\n생 양고기 살치\n26,000..."
3,따띠따띠,4.6,"19,347",15,680,퓨전 양식
4,무아,4.6,"16,614",12,578,"[퓨전 일식, 디너코스\n110,000원]"
5,상석,4.6,"15,415",8,447,회 / 스시
6,레우레우,4.6,"1,836",9,80,태국 음식
7,쏭타이,4.6,"53,482",22,"1,109","[태국 음식, 쏨탐\n14,000원\n텃만꿍 (4ea)\n15,000원\n팟카파오무..."
8,히노카츠,4.6,"4,607",7,127,까스 요리
9,파씨오네,4.5,"531,470",150,"10,423",프랑스 음식
